In [1]:
!python3 -m pip install paddlepaddle-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00


In [2]:
!pip install "paddleocr>=2.0.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=717bc375a5b472dad1621d0d3561cf82c1b7f1602ed21da4575fec8e4db85eaa
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [3]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 18.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import easyocr  # EasyOCR uses PyTorch under the hood for GPU acceleration
import torch
import os
import pandas as pd

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
reader = easyocr.Reader(['en'], gpu=(device == 'cuda'))  # Use GPU if available

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Function to load image from URL
def load_image_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content))
        image_np = np.array(image)
        return image_np
    except Exception as e:
        print(f"Failed to load image from {url}: {e}")
        return None

# Function to process a single image and extract text using EasyOCR
def process_image(image_link):
    img = load_image_from_url(image_link)
    if img is not None:
        # EasyOCR accepts file paths or numpy arrays
        result = reader.readtext(img)
        finaltext = ' '.join([text[1] for text in result])  # Extract text from OCR result
        return image_link, finaltext
    return image_link, ''  # Return empty text if the image failed to load

# Function to save checkpoint
def save_checkpoint(df, batch_number, checkpoint_dir='/content/drive/My Drive/checkpoints/'):
    # if not os.path.exists(checkpoint_dir):
    #     os.makedirs(checkpoint_dir)
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_batch_{batch_number}.csv')
    df.to_csv(checkpoint_path, index=False)
    print(f"Checkpoint saved: {checkpoint_path}")

# Function to process images in parallel
def process_images_in_parallel(image_links, max_workers=10):
    linkd = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use a ThreadPool to parallelize processing
        results = list(tqdm(executor.map(process_image, image_links), total=len(image_links), desc="Processing images"))

    # Collect results into a dictionary
    for image_link, text in results:
        linkd[image_link] = text
    return linkd

In [6]:
# Main processing function with checkpointing
def main(df, checkpoint_dir='/content/drive/My Drive/checkpoints/', max_workers=10, batch_size=2000):
    # Ensure unique image links
    unique_image_links = df['image_link'].unique()

    # Load the last checkpoint (if it exists)
    checkpoint_files = sorted([f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_batch_')])
    last_batch_processed = 0

    if checkpoint_files:
        last_checkpoint_file = checkpoint_files[-1]
        last_batch_processed = int(last_checkpoint_file.split('_')[-1].replace('.csv', ''))
        print(f"Resuming from checkpoint: {last_checkpoint_file}")
        df = pd.read_csv(os.path.join(checkpoint_dir, last_checkpoint_file))

    linkd = {}

     # Process images in batches, starting from the last checkpoint
    for i in range(last_batch_processed + 1, len(unique_image_links), batch_size):
        batch_links = unique_image_links[i:i+batch_size]
        batch_result = process_images_in_parallel(batch_links, max_workers=max_workers)
        df['description'].update(pd.Series(batch_result))  # Update df with new descriptions

        # Save progress after each batch
        save_checkpoint(df, i, checkpoint_dir=checkpoint_dir)

    return df

In [7]:
df= pd.read_csv('/content/train.csv')

In [8]:
import time
start_time= time.time()

In [9]:
!ls /content/drive/My\ Drive

 09-2-M2020-CP-Functions-More.pptx
'ACS- Unit 4 - Part 2 - slides (1).gdoc'
'ACS- Unit 4 - Part 2 - slides (1).pdf'
'ADA class on 25.02.22 at 2-3 PM (rko-vkwd-kcp - Feb 25, 2022).gdoc'
 ada.docx
 ada.zip
 Assessment.gform
 avlsi.docx
 avlsi_try1.gdoc
 avlsi_try2.gdoc
 B2_U2_Assignment_S20210020265.docx
'Batch 29.pdf'
'Batch 29.zip'
 bhavya.docx
'Bird Strikes data.xlsx - Bird Strikes.csv'
'CCN ASSIGNMENT.pdf'
'ccn.cpp - cpp - Visual Studio Code 2023-04-14 10-38-34.mp4'
 certificate.pdf
 checkpoints
 ChrisAryan_S20210020265_assn1.ipynb
'CHRIS ARYAN S20210020265 OCW LAB.docx'
 chriscv.pdf
'Chris_resume (1).pdf'
 Chris_resume.pdf
'Chris_S20210020265 (1).pdf'
'Chris_S20210020265 (2).pdf'
 Chris_S20210020265.pdf
 Classroom
'Colab Notebooks'
'Copy of Mutual_Consent_form_BTP.docx'
'Copy of Mutual_Consent_form_BTP.gdoc'
 data
 DIP_assn1.ipynb
'EEN_A1_S20210020265_Chris Aryan.pdf'
'EEN_A2_S20210020265_Chris Aryan.pdf'
'EXPERIMENT 2.pdf'
'EXPERIMENT 5 HDL.pdf'
 Extraction_data_2023_august_to_dece

In [10]:
checkpoint_dir = '/content/drive/My Drive/checkpoints/'

# Check if the directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print(f"Created directory: {checkpoint_dir}")
else:
    print(f"Directory already exists: {checkpoint_dir}")


Directory already exists: /content/drive/My Drive/checkpoints/


In [29]:
!ls /content/drive/My\ Drive

 09-2-M2020-CP-Functions-More.pptx
'ACS- Unit 4 - Part 2 - slides (1).gdoc'
'ACS- Unit 4 - Part 2 - slides (1).pdf'
'ADA class on 25.02.22 at 2-3 PM (rko-vkwd-kcp - Feb 25, 2022).gdoc'
 ada.docx
 ada.zip
 Assessment.gform
 avlsi.docx
 avlsi_try1.gdoc
 avlsi_try2.gdoc
 B2_U2_Assignment_S20210020265.docx
'Batch 29.pdf'
'Batch 29.zip'
 bhavya.docx
'Bird Strikes data.xlsx - Bird Strikes.csv'
'CCN ASSIGNMENT.pdf'
'ccn.cpp - cpp - Visual Studio Code 2023-04-14 10-38-34.mp4'
 certificate.pdf
 checkpoints
 ChrisAryan_S20210020265_assn1.ipynb
'CHRIS ARYAN S20210020265 OCW LAB.docx'
 chriscv.pdf
'Chris_resume (1).pdf'
 Chris_resume.pdf
'Chris_S20210020265 (1).pdf'
'Chris_S20210020265 (2).pdf'
 Chris_S20210020265.pdf
 Classroom
'Colab Notebooks'
'Copy of Mutual_Consent_form_BTP.docx'
'Copy of Mutual_Consent_form_BTP.gdoc'
 data
 DIP_assn1.ipynb
'EEN_A1_S20210020265_Chris Aryan.pdf'
'EEN_A2_S20210020265_Chris Aryan.pdf'
'EXPERIMENT 2.pdf'
'EXPERIMENT 5 HDL.pdf'
 Extraction_data_2023_august_to_dece

In [ ]:
df = main(df, max_workers=10, batch_size=2000)  # Adjust `max_workers` and `batch_size` based on system's resources
print(f"Completed processing in {time.time() - start_time} seconds")
df.head()

Processing images:   0%|          | 0/2000 [00:00<?, ?it/s]